In [1]:
import os
from dotenv import load_dotenv

In [3]:
try:
    load_dotenv()
    print("Key successfully imported")
except Exception as e:
    print("failed to load the env file")

Key successfully imported


In [6]:
try:
    GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Authentication successful for the key")
except Exception as e:
    print(f"Error loading the key {e}")

Authentication successful for the key


# Importing Google's ADK components


In [7]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("all imports were successful")

all imports were successful


# Retry options

if the process fails to run, a backup which would retry on it's own

In [11]:
retry_config = types.HttpRetryOptions(
    attempts = 5,
    exp_base = 7,
    initial_delay = 1,
    http_status_codes = [429,500,503,504],
)
print("retry config working")

retry config working


## Section - Why Multi-Agent Systems? + Your First multi-agent

### Why we need multi agents - "the do it all agent"

Single agents can do a lot. But what happens when the task gets complex? A single "monolithic" agent that tries to do research, writing, editing, 
and fact-checking all at once becomes a problem. Its instruction prompt gets long and confusing. It's hard to debug (which part failed?), difficult to maintain, and often produces unreliable results.

### How we are solving the problem - "A team of specialists"

Instead of one "do-it-all" agent, we can build a multi-agent system. This is a team of simple, specialized agents that collaborate, just like a real-world team. Each agent has one clear job (e.g., one agent only does research, another only writes). This makes them easier to build, easier to test, and much more powerful and reliable when working together.

### Architecture

<img src = "/Users/mario/Desktop/kaggle/Day-1-B/1.png"/>
<img src = "/Users/mario/Desktop/kaggle/Day-1-B/2.png"/>




# 2.1 Research Specialization Assistant - Research and Summarization system

A system with two agents:-
1) Research agent - Searches using Google search
2) Summarizer agent - Creates concise summaries from research findings

In [12]:
# research agent 
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.
    """,
    tools=[google_search],
    output_key="research_findings" # the result of this agent will be stored in the session state with this key
)

print("research agent created.")

research agent created.


In [14]:
#summarizer agent 
summarizer_agent = Agent(
    name="SummarizerAgent",
    model = Gemini(
        name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    Read the provided research findings: {research_findings}. Create a concise summary as a bulleted list with 3-5 key points.
    """,
    output_key = "final_summary"
)
print("summarizer agent created")

summarizer agent created


In [19]:
# then we bring the agents under a root agent or a coordinator agent
# below we are wrapping the sub agents in "AgentTool" to make them callable 
# for the root agent

In [18]:
try:
    root_agent = Agent(
        name="RootAgent",
        model=Gemini(
            model="gemini-2.5-flash-lite",
            retry_options = retry_config,
        ),
        instruction=""" You are a research coordinator. Your goal is to answer a user's query by orchestrating a workflow
        1. First you must call the `ResearchAgent` tool to find the relevant information on the topic provided by the user.
        2. Next, after receiving the research findings, you must call `SummarizerAgent` to create a concise summary.
        3. Fiannly, present the final summary clearly to the user as your response""",
        tools = [AgentTool(research_agent), AgentTool(summarizer_agent)],
    )
    print("root agent created")
except Exception as e:
    print(f"Error creating the agent {e}")

root agent created


In [21]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(" Who are the members of PinkFloyd, where are the located right now ")


 ### Created new session: debug_session_id

User >  Who are the members of PinkFloyd, where are the located right now 


RootAgent > The surviving members of Pink Floyd are David Gilmour, Nick Mason, and Roger Waters. David Gilmour is located in England and is touring globally in late 2024 for his new solo album, "Luck and Strange." Nick Mason resides in Wiltshire, England, and is actively touring with his band, Nick Mason's Saucerful of Secrets. Roger Waters is an extensively touring musician who released "Dark Side of the Moon Redux" in 2023; his exact current location is not specified.


# Section 3 - sequential workflows - The Assembly Line

## The Problem: Unpredictable Order

The previous multi-agent system worked, but it relied on a detailed instruction prompt to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

## The Solution: A Fixed Pipeline

When you need tasks to happen in a guaranteed, specific order, you can use a SequentialAgent. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

### Use Sequential when: Order matters, you need a linear pipeline, or each step builds on the previous one.

Architecture: blog creation pipeline

<img src="/Users/mario/Desktop/kaggle/Day-1-B/3.png"/>



# Blog post creation with sequential agent
1) OUtline agent - creates a blog outline for a given topic
2) Writer agent - writes a blog post
3) Editor agent - Edits a blog post draft for clarity and structure

In [29]:
# OUtline agent - creates an outline agent for blog 
try:
    outline_agent = Agent(
        name="OutlineAgent",
        model=Gemini(
            model = "gemini-2.5-flash-lite",
            retry_options = retry_config
        ),
        instruction=
        """
        You are a specialized writer with a well over experience of writing blogs for more than
        20+ years. you are supposed to make blogs in the how Rick talks to morty, it should match the exact tone
        of how Rick responds, the blog should reflect as if it's Rick who's written it keeping these points in mind:
        1) A Catchy Headline
        2) An Introduction hook
        3) 3-5 main sections with 2-3 bullet points for each
        4) A concluding thought
        """,
        output_key="blog_outline", # the result of the agent will be stored in the session state with the key.

    )
    print("outline agent created")
except Exception as e:
    print(f"The agent failed to create due to {e}")

outline agent created


In [30]:
# Writer agent - writes full blog post based on the outline from teh previous agent

writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=""" Following this outline strictly: {blog_outline}
    write a blog, 300-400 word post with an engaging and informative tone.""",
    output_key = "blog_draft"
)
print("writer agent created")

writer agent created


In [31]:
# Editor agent - edits and polishes the draft from teh writer agent
editor_agent=Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow 
    and sentence structure , enhancing the overall clarity and making it close to How Rick actaully speaks,
    you have to make sure to break the 4th wall. 
    """,
    output_key="final_blog"
)
print("editor_agent created")

editor_agent created


In [32]:
# then we bring all the agents under a sequential agent, which runs the agents in 
# order that they are listed:
try:
    root_agent = SequentialAgent(
        name="BlogPipeline",
        sub_agents=[outline_agent, writer_agent, editor_agent],
    )
    print("the pipeline runs successfully")
except Exception as e:
    print(f"The pipeline is not running due to this error {e}")

the pipeline runs successfully


# now time to test the sequential agents

In [33]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(" Write a blog about AC/DC and black sabbath comparing the two and what they each give. ")


 ### Created new session: debug_session_id

User >  Write a blog about AC/DC and black sabbath comparing the two and what they each give. 
OutlineAgent > Alright, listen up, you jabronis! Rick here, and I'm about to drop some sonic knowledge bombs on your puny brains. Today, we're diving headfirst into the primordial ooze of heavy rock, pitting two titans against each other: AC/DC and Black Sabbath. Don't get me wrong, they're both loud enough to shatter galaxies, but they bring their own brand of interdimensional mayhem to the table. So strap in, grab a flask, and let's see who's the real riff master.

## AC/DC vs. Black Sabbath: Which Butt-Rocking Behemoth Reigns Supreme?

You think you know rock and roll? Pffft. You probably still think listening to The Beach Boys is edgy. Well, let me tell you, these two bands are the real deal. They're not here to serenade your grandma; they're here to melt your face off and leave you begging for more. So, what's the difference, you ask? It's lik

# Section 4 - Parallel Workflows - independent Researchers

# The Problem: The Bottleneck

The previous sequential agent is great, but it's an assembly line. Each step must wait for the previous one to finish. What if you have several tasks that are not dependent on each other? For example, researching three different topics. Running them in sequence would be slow and inefficient, creating a bottleneck where each task waits unnecessarily.

# The Solution: Concurrent Execution

When you have independent tasks, you can run them all at the same time using a ParallelAgent. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

Use Parallel when: Tasks are independent, speed matters, and you can execute concurrently.

# Architecture

<img src = "/Users/mario/Desktop/kaggle/Day-1-B/4.png"/>

# 4.1 Example - Parallel Multi topic research agent
building a system with 4 agents

1) Tech Researcher - Researches AI/Ml news and trends
2) Health Researcher - Researches recent medical news and trends
3) Finance Researcher - Researches finance and fintech news and trends
4) Aggregator agent - combines all teh findings into a single summary

In [41]:
# tech_researcher 
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options= retry_config
    ),
    instruction="""
    Research the latest AI/ML trends. Include 3 key developments, the main companies involved, and the potential impact. Keep the report very concise (100 words).
    """,
    tools=[google_search],
    output_key="tech_researcher",
)
print("Researcher created")

Researcher created


In [42]:
# health researcher
health_researcher=Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
tools=[google_search],
output_key="health_researcher"
)
print("agent created")

agent created


In [43]:
# finance reseacrher
finance_researcher=Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = """ Make a thorough research about the Market and fintech trends
    you have to consider all the dynamics of how external factors affect the trend. look at the data of past week and then select 4 highest trending stocks
    and then based on history, news, posts, articles give your verdict about the trend selected stocks would see either rise or fall and then make sure to lookout for future
    Make sure to include 3 key trends, market implications and make a report (200 words) """,
    tools=[google_search],
    output_key="finance_researcher"
)
print("agent created")

agent created


# The aggregator agent

In [44]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = """Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_researcher}
    
    **Health Breakthroughs:**
    {health_researcher}
    
    **Finance Innovations:**
    {finance_researcher}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",
)
print("aggregator agent created")

aggregator agent created


# 
👉 **Then we bring the agents together under a parallel agent, which is itself nested inside of a sequential agent.**

This design ensures that the research agents run first in parallel, then once all of their research is complete, the aggregator agent brings together all of the research findings into a single report:


In [45]:
# paraller agent
parallel_research_agent=ParallelAgent(
    name="ParallelResearchAgent",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="RootAgent",
    sub_agents=[parallel_research_agent, aggregator_agent],
)
print("Parallel and Sequential agent created")

Parallel and Sequential agent created


In [46]:
try:
    runner = InMemoryRunner(agent=root_agent)
    response = await runner.run_debug("Run the daily executive briefing on Tech, Health, and Finance")
except Exception as e:
    print(f"Error in some code {e}")


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
HealthResearcher > **Health:** Gene therapy is rapidly advancing, with recent breakthroughs enabling a deaf child to hear and restoring vision. CRISPR-based therapies are also showing promise for blood disorders like sickle cell anemia. These treatments are moving from experimental to approved, with wider applications anticipated within 1-5 years.

**Technology:** Agentic AI, capable of autonomous planning and action, is projected to handle 15% of enterprise decisions by 2028. This will significantly reduce manual work and reshape industries. The development of domain-specific AI models and AI-native platforms also promises cost reductions and improved ROI. Widespread adoption is expected within the next 1-4 years.

**Finance:** Innovations in AI and machine learning are revolutionizing risk assessment and user experience. Blockchain and decentralized finance (DeFi) continu

So far, all our workflows run from start to finish and then stop. But what if you need to review and improve an output multiple times? Next, we'll build a workflow that can loop and refine its own work.

# Section 5: Loop workflows - The Refinement Cycle

# The Problem: One-Shot Quality

All the workflows we've seen so far run from start to finish. The SequentialAgent and ParallelAgent produce their final output and then stop. This 'one-shot' approach isn't good for tasks that require refinement and quality control. What if the first draft of our story is bad? We have no way to review it and ask for a rewrite.

# The Solution: Iterative Refinement

When a task needs to be improved through cycles of feedback and revision, you can use a LoopAgent. A LoopAgent runs a set of sub-agents repeatedly until a specific condition is met or a maximum number of iterations is reached. This creates a refinement cycle, allowing the agent system to improve its own work over and over.

# Use Loop when: Iterative improvement is needed, quality refinement matters, or you need repeated cycles.


Architecture: Story Writing & Critique Loop

<img src="/Users/mario/Desktop/kaggle/Day-1-B/5.png"/>

In [48]:
# Iterative Story Refinement
# we will build a system - a system with two agent
# 1) Writer Agent - Writes a draft of a short story
# 2) Critic Agent - Reviews and Critiques the story to suggest improvements

In [49]:
initial_writer = Agent(
    name="InitialWriter",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options = retry_config,
    ),
    instruction = """Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story"
)
print("INitial draft agent created")

INitial draft agent created


In [50]:
# this agent only critiques the work of the previous agent, gives feedback or the approval signal. it has no tools
critique_agent = Agent(
    name="CritiqueAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = """You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique"
)
print("Critique agent done")

Critique agent done


Now, we need a way for the loop to actually stop based on the critic's feedback. The LoopAgent itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

A simple Python function that the LoopAgent understands as an "exit" signal.
An agent that can call that function when the right condition is met.
First, you'll define the exit_loop function:

In [51]:
def exit_loop():
    """ Call this function only when the critique is 'APPROVED', indicating the story is finished and no more changes
    are needed."""
    return {"status":"approved", "message":"Story approved. Exiting refinement loop."}
print("exit loop function created.")

exit loop function created.



To let an agent call this Python function, we wrap it in a FunctionTool. Then, we create a RefinerAgent that has this tool.

👉 Notice its instructions: this agent is the "brain" of the loop. It reads the {critique} from the CriticAgent and decides whether to (1) call the exit_loop tool or (2) rewrite the story.

In [52]:
# this agent is responsible for the refinement of the story based on critique or calls the exit loop function
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    instruction = """You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key = "current_story",
    tools = [FunctionTool(exit_loop)], # we have initialized our tool with correct function reference
)
print("refiner_agent created")

refiner_agent created


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of max_iterations:

In [54]:
# the loop agent contains the agent that will run repeatedly: critic -> refiner
story_refinement_loop = LoopAgent(
    name="StoryRefinementAgent",
    sub_agents = [critique_agent, refiner_agent],
    max_iterations = 3 # prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents = [initial_writer, story_refinement_loop],

)
print("loop and sequential agents created")

loop and sequential agents created


In [55]:
runner = InMemoryRunner(agent = root_agent)
response = await runner.run_debug("IN a galaxy far far away where anakin was a farmer")


 ### Created new session: debug_session_id

User > IN a galaxy far far away where anakin was a farmer
InitialWriter > Anakin Skywalker, not a Jedi Knight, but a moisture farmer on Tatooine, squinted against the twin suns. The dry wind whipped sand against his goggles. His hands, usually steady on a lightsaber, were calloused from wrestling with stubborn vaporators. Today, a familiar, gnawing unease settled in his gut. He’d had dreams again, strange flashes of power and darkness, a voice whispering of destiny. He dismissed them as the heat playing tricks. He had crops to tend, not galactic empires to conquer. Still, as he trudged back to his small homestead, a flicker of something – impatience, perhaps, or a nascent rage – sparked within him, a seed waiting for the right kind of soil to bloom.
CritiqueAgent > The story presents an intriguing premise, but needs some development. Here are a few suggestions:

1.  **Elaborate on the "dreams":** The "strange flashes of power and darkness" a

CritiqueAgent > APPROVED


# Section 6 - Choosing the RIGHT pattern

Decision tree: which workflow pattern?

<img src="/Users/mario/Desktop/kaggle/Day-1-B/6.png"/>

# quick reference table 
<img src="/Users/mario/Desktop/kaggle/Day-1-B/7.png"/>